In [1]:
import pandas as pd

from pandarallel import pandarallel
from rdkit import Chem

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
df = pd.read_csv('../data/surechembl_smiles_canon_chiral_randomized.csv')
# df.rename(columns={"SMILES":"smiles"})
# df = dd.rename(columns={'SMILES': 'smiles'})

In [3]:
tmp_df = df[:10]

In [4]:
def func(smi):
    try:
        mol = Chem.MolFromSmiles(smi)
        return Chem.MolToInchiKey(mol)
    except:
        return None

In [5]:
tmp_df['inchi_key'] = tmp_df['SMILES'].parallel_apply(func)

/var/folders/67/6sj3_z4943v4gy9hc27k5vv80000gq/T/ipykernel_20424/1386816321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['inchi_key'] = tmp_df['SMILES'].parallel_apply(func)


In [6]:
# load in "../data/CID-InChI-Key" as a dictionary
inchi_to_cid = {}
with open('../data/CID-InChI-Key') as f:
    for line in f:
        (val, _, key) = line.split()
        inchi_to_cid[key] = val

In [7]:
# parallel map inchi_key to cid
tmp_df['cid'] = tmp_df['inchi_key'].parallel_apply(lambda x: inchi_to_cid.get(x, None))

/var/folders/67/6sj3_z4943v4gy9hc27k5vv80000gq/T/ipykernel_20424/4157399715.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['cid'] = tmp_df['inchi_key'].parallel_apply(lambda x: inchi_to_cid.get(x, None))


In [12]:
from collections import defaultdict

cid_to_patents = defaultdict(set)

with open('../data/CID-Patent') as f:
    for line in f:
        (cid, patent) = line.split()
        # there may be many lines that map to the same cid
        # so we need to append to a list
        cid_to_patents[cid].update(patent)


In [ ]:
# tmp_df["patent_ids"] = tmp_df["cid"].parallel_apply(lambda x: cid_to_patents.get(x, None))

In [ ]:
# tmp_df